In [1]:
# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Training utils
import utils_svr

# optuna
import optuna

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

c:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates = ['FECHAHORA'])

In [3]:
estacion = 4

variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 
             'TEMPERATURA', 'HUMEDAD', 'PRESION', 'TEMPERATURA_PRONOSTICO', 
             'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO', 'DIA_SEM', 'TRAFICO' , 'AQI_MP10', 'AQI_MP2_5']

dependent = ['AQI_MP2_5']

number_of_features = len(variables)

training_days = 7 
forecast_days = 1 
samples_per_day = 288
step = 288/4

# Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
train_months = relativedelta(months = 12)

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

In [4]:
datos['AQI_MP2_5'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1315882 entries, 0 to 1315881
Series name: AQI_MP2_5
Non-Null Count    Dtype  
--------------    -----  
1315882 non-null  float64
dtypes: float64(1)
memory usage: 10.0 MB


In [5]:
%%time

X_train, y_train, X_test, y_test = utils_svr.get_everything(datos, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)

CPU times: total: 15.7 s
Wall time: 15.8 s


In [6]:
params = {
    'kernel'    : 'rbf',
    'gamma'     : 'auto',
    'C'         : 0.1,
    'epsilon'   : 1,
    'cache_size': 4000
    }   

optuna_svr_model = SVR(** params)

trained_svr = MultiOutputRegressor(optuna_svr_model).fit(X_train, y_train)



In [7]:
y_pred = trained_svr.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)

print('MAPE')
print(mape)
print('MAE')
print(mean_absolute_error(y_pred, y_test))
print('RMSE')
print(mean_squared_error(y_pred, y_test, squared= False))

MAPE
0.7045955073827199
MAE
11.454504941648004
RMSE
15.156805696288947


In [8]:
def objective(trial):
    
    params = {
    'kernel'    : 'rbf',
    'gamma'     : trial.suggest_loguniform('gamma', 1e-8, 1.0),
    'C'         : trial.suggest_loguniform('C', 1e-5, 100.0),
    'epsilon'   : 1,
    'cache_size': 4000
    }   

    optuna_svr_model = SVR(** params)

    trained_svr = MultiOutputRegressor(optuna_svr_model).fit(X_train, y_train)

    y_pred = trained_svr.predict(X_test)

    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    print('MAPE')
    print(mape)
    print('MAE')
    print(mean_absolute_error(y_pred, y_test))
    print('RMSE')
    print(mean_squared_error(y_pred, y_test, squared= False))
    
    
    return mape

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials = 30)

trial = study.best_trial

import joblib

joblib.dump(study, "study_SVR_e4_MP2.5_correcto_7_days.pkl")

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-02-05 15:13:01,371] A new study created in memory with name: no-name-722c8244-db08-45e0-992b-5c70354369f6


MAPE
0.6800266775492424
MAE
11.158592580039867
RMSE
14.888419810842898


[I 2023-02-05 15:24:49,980] Trial 0 finished with value: 0.6800266775492424 and parameters: {'gamma': 3.0876887454597824e-06, 'C': 0.02966993854745562}. Best is trial 0 with value: 0.6800266775492424.


MAPE
0.4721396841880735
MAE
8.655067712709338
RMSE
12.443032773539796


[I 2023-02-05 15:37:19,753] Trial 1 finished with value: 0.4721396841880735 and parameters: {'gamma': 3.345841696928003e-07, 'C': 1.2548791130945498}. Best is trial 1 with value: 0.4721396841880735.


...

MAPE
0.3588288525566349
MAE
6.845006700409575
RMSE
9.881475249689663


[I 2023-02-06 05:39:56,140] Trial 28 finished with value: 0.3588288525566349 and parameters: {'gamma': 1.7498972177030344e-07, 'C': 88.76229687780621}. Best is trial 28 with value: 0.3588288525566349.


MAPE
0.5460966812892696
MAE
9.578190298455317
RMSE
13.460289956188525


[I 2023-02-06 07:36:34,765] Trial 29 finished with value: 0.5460966812892696 and parameters: {'gamma': 3.949427233085567e-06, 'C': 59.681821819798884}. Best is trial 28 with value: 0.3588288525566349.


    gamma: 1.7498972177030344e-07
    C: 88.76229687780621
